# Extracting player info and saving into SQL Database

This file creates a database in sqlite by pulling out information for female professional tennis players from 2010 - 2020. More specifically, this database largely relies on data from www.tennisexplorer.com to extract match, injury, and rank history. In addition, basic demographic data was recorded for each player when present. Data was pulled specifically from the top 200 players from 2010 - 2020.
<br><br>__The following tables will be created in this database:__
* __Players__ (name, birthday, current/highest singles and doubles ranking, hand, sex, height, and weight)
* __Injury_Record__ (name_id,start_date,end_date, tournament,reason )
* __Match_Record__ (name_id, year, tournament, match_type, date, opponent, round, result, h ,a)
* __Ranks__ (name_id, year, ranktype, rank
* __Court_Type__ (Tournament, Court Type)

In [162]:
# Loading Required Packages
import pprint
from bs4 import BeautifulSoup
import requests
from urllib import request, response, error, parse
from urllib.request import urlopen
import time
import pandas as pd
import random
import os
import sqlite3
import random

In [163]:
# Change the Working Directory
os.chdir("/Users/Owner/Desktop/InsightFellows/Daniella_Patton_Insight_Project/Raw_Data")

In [164]:
# connecting to sqlite
conn = sqlite3.connect('tennis_players.sqlite')
cur = conn.cursor()

# Basic Data Table

In [165]:
# Makeing the Players Table
cur.executescript('''
DROP TABLE IF EXISTS Players;

CREATE TABLE Players (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE,
    birthday    TEXT,
    country    TEXT,
    singles_curr_rank    INTEGER,
    singles_high_rank    INTEGER,
    doubles_curr_rank    INTEGER,
    doubles_high_rank    INTEGER,
    hand    TEXT,
    sex    TEXT,
    weight    TEXT,
    height    TEXT
);

''')

## Basic Data Function: 
For each player, the following basic information is saved:<br>
Age, height, weight, birthday, country, singles/doubles ranking, year
<br>
<br> __Input__ - The web-page from www.tennisexplorer.com and year of interest
<br>__Output__ - Adding rows to the Player sqlite table

In [166]:
def basic_data(soup, year):
    x = 0
    name = soup.title # name
    name = name.text[:-18]
    #names.append(name.text[:-18])
    for i, table in enumerate(soup.select("table")):
        # print(i, table['class'])
        # Basic Data
        if table["class"] == ["plDetail"]:
            rows = table.findAll("div", {"class": "date"})
            for row in rows:
                row = row.text
                x = x + 1
                #print(x)
                # Country
                if "Country" in row:
                    country = row[9:] # country
                # Birthday
                if "Age" in row:
                    birthday = row
                    birthday = birthday[birthday.find("(") + 1 : birthday.find(")")] # birthday
                # Singles Rankings
                if "Current/Highest rank - singles" in row:
                    sin_ranks = row
                    cur_rank = sin_ranks.split(" / ")[0]
                    cur_rank_singles = cur_rank.split("singles: ")[1] # singles_curr_rank
                    if cur_rank_singles == '-':
                        cur_rank_singles = None
                    else: 
                        cur_rank_singles = int(float(cur_rank_singles))    
                    hi_rank_singles = float(sin_ranks.split(" / ")[1]) # singles_high_rank
                # Doubles Ranking
                if "Current/Highest rank - doubles" in row:
                    dub_ranks = row
                    cur_rank = dub_ranks.split(" / ")[0]
                    cur_rank_dubs = cur_rank.split("doubles: ")[1] # doubles_curr_rank
                    if cur_rank_dubs == '-':
                        cur_rank_dubs = None
                    else: 
                        cur_rank_dubs = int(float(cur_rank_dubs))  
                    hi_rank_dubs = int(float(dub_ranks.split(" / ")[1])) # doubles_high_rank
                # Sex
                if "Sex" in row:
                    s = row.split("Sex: ")
                    sex = s[1] # sex
                # Handed
                if "Plays" in row:
                    h = row
                    hand = h.split("Plays: ")[1] # hand
                # Height & Weight
                if "Height / Weight" in row:
                    hw = row[17:]
                    height = str(hw.split(" / ")[0]) # height
                    weight = str(hw.split(" / ")[1]) # weight       
                elif "Weight" in row:
                    weight = str(row[8:]) # weight
              
            try:
                height
            except NameError:
                height = None
            try:
                weight
            except NameError:
                weight = None
            try:
                cur_rank_dubs
            except NameError:
                cur_rank_dubs = None
            try:
                hi_rank_dubs
            except NameError:
                hi_rank_dubs = None 

    cur.execute('''INSERT OR IGNORE INTO Players 
    ( name, birthday, country, singles_curr_rank, singles_high_rank, doubles_curr_rank, doubles_high_rank,
        hand, sex, weight, height)
        VALUES ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? )''', 
                ( str(name), str(birthday), str(country),
                 cur_rank_singles, int(hi_rank_singles),
                 cur_rank_dubs, hi_rank_dubs,
                 str(hand), str(sex), weight, height ))
    conn.commit()  

## Loop
This loop pulls out player-specific information from various years.

In [167]:
# Creating SQLite table for Basic Data
for i in range(0, 11):
    """Basic Data Loop
    For each player, the following basic information is saved:
    Age, height, weight, birthday, country, singles/doubles ranking, year
    """
    year = 2020 - i
    url_b = "https://www.tennisexplorer.com/ranking/wta-women/" + str(year)

    for i in range(1, 5):
        url = url_b + "/?page=" + str(i)
        with requests.get(url) as r:
            soup = BeautifulSoup(r.content, features="html")
            sl = random.randint(1, 3)
            time.sleep(sl)

        data = soup.find("tbody", {"class": "flags"})
        rows = data.find_all("td", {"class": "t-name"})

        for t in rows:
            player = t.find("a").attrs["href"]
            player_url = "https://www.tennisexplorer.com/" + player[1:]
            print(player_url)

            with requests.get(player_url) as r:
                soup = BeautifulSoup(r.content, features="html")
                basic_data(soup, year)
                sl = random.randint(1, 3)
                time.sleep(sl)

https://www.tennisexplorer.com/player/barty/
https://www.tennisexplorer.com/player/halep/
https://www.tennisexplorer.com/player/pliskova/
https://www.tennisexplorer.com/player/kenin-5a3a8/
https://www.tennisexplorer.com/player/svitolina/
https://www.tennisexplorer.com/player/andreescu-281c7/
https://www.tennisexplorer.com/player/bertens/
https://www.tennisexplorer.com/player/bencic/
https://www.tennisexplorer.com/player/williams-3d2a9/
https://www.tennisexplorer.com/player/osaka-fa603/
https://www.tennisexplorer.com/player/sabalenka/
https://www.tennisexplorer.com/player/kvitova/
https://www.tennisexplorer.com/player/keys/
https://www.tennisexplorer.com/player/konta/
https://www.tennisexplorer.com/player/martic/
https://www.tennisexplorer.com/player/muguruza-33acd/
https://www.tennisexplorer.com/player/rybakina/
https://www.tennisexplorer.com/player/vondrousova/
https://www.tennisexplorer.com/player/riske-69d46/
https://www.tennisexplorer.com/player/sakkari/
https://www.tennisexplorer.

https://www.tennisexplorer.com/player/danilovic-ae861/
https://www.tennisexplorer.com/player/frech/
https://www.tennisexplorer.com/player/paquet/
https://www.tennisexplorer.com/player/kucova-b6760/
https://www.tennisexplorer.com/player/xun/
https://www.tennisexplorer.com/player/bucsa/
https://www.tennisexplorer.com/player/you-fe146/
https://www.tennisexplorer.com/player/podoroska/
https://www.tennisexplorer.com/player/ruse/
https://www.tennisexplorer.com/player/rakhimova-29cc5/
https://www.tennisexplorer.com/player/lepchenko/
https://www.tennisexplorer.com/player/schmiedlova/
https://www.tennisexplorer.com/player/rybarikova/
https://www.tennisexplorer.com/player/bolkvadze/
https://www.tennisexplorer.com/player/buyukakcay/
https://www.tennisexplorer.com/player/lu-ce849/
https://www.tennisexplorer.com/player/de-vroome/
https://www.tennisexplorer.com/player/shinikova/
https://www.tennisexplorer.com/player/schoofs/
https://www.tennisexplorer.com/player/han-43b12/
https://www.tennisexplorer

https://www.tennisexplorer.com/player/martincova-a08dc/
https://www.tennisexplorer.com/player/arconada-956c5/
https://www.tennisexplorer.com/player/jorovic/
https://www.tennisexplorer.com/player/osuigwe/
https://www.tennisexplorer.com/player/bacsinszky/
https://www.tennisexplorer.com/player/nara-53cea/
https://www.tennisexplorer.com/player/minella/
https://www.tennisexplorer.com/player/dart/
https://www.tennisexplorer.com/player/kiick/
https://www.tennisexplorer.com/player/wang-d78f7/
https://www.tennisexplorer.com/player/friedsam/
https://www.tennisexplorer.com/player/dolehide-903f7/
https://www.tennisexplorer.com/player/haas-ae7b6/
https://www.tennisexplorer.com/player/tomova/
https://www.tennisexplorer.com/player/li-f8666/
https://www.tennisexplorer.com/player/wang-77549/
https://www.tennisexplorer.com/player/cepede-royg/
https://www.tennisexplorer.com/player/wickmayer/
https://www.tennisexplorer.com/player/kostyuk-ea2bf/
https://www.tennisexplorer.com/player/gatto-monticone/
https:

https://www.tennisexplorer.com/player/vickery/
https://www.tennisexplorer.com/player/ferro/
https://www.tennisexplorer.com/player/minella/
https://www.tennisexplorer.com/player/golubic/
https://www.tennisexplorer.com/player/safarova/
https://www.tennisexplorer.com/player/kuznetsova/
https://www.tennisexplorer.com/player/kalinina/
https://www.tennisexplorer.com/player/danilovic-ae861/
https://www.tennisexplorer.com/player/zvonareva/
https://www.tennisexplorer.com/player/duque-marino/
https://www.tennisexplorer.com/player/pegula/
https://www.tennisexplorer.com/player/errani/
https://www.tennisexplorer.com/player/siegemund/
https://www.tennisexplorer.com/player/hibino/
https://www.tennisexplorer.com/player/brady-cbcc6/
https://www.tennisexplorer.com/player/zhuk-8e659/
https://www.tennisexplorer.com/player/korpatsch/
https://www.tennisexplorer.com/player/rus-a3117/
https://www.tennisexplorer.com/player/kostyuk-ea2bf/
https://www.tennisexplorer.com/player/zhu-a98e2/
https://www.tennisexplor

https://www.tennisexplorer.com/player/vondrousova/
https://www.tennisexplorer.com/player/osaka-fa603/
https://www.tennisexplorer.com/player/siegemund/
https://www.tennisexplorer.com/player/riske-69d46/
https://www.tennisexplorer.com/player/haddad-maia/
https://www.tennisexplorer.com/player/linette/
https://www.tennisexplorer.com/player/sabalenka/
https://www.tennisexplorer.com/player/bencic/
https://www.tennisexplorer.com/player/watson-4ffc1/
https://www.tennisexplorer.com/player/flipkens/
https://www.tennisexplorer.com/player/van-uytvanck/
https://www.tennisexplorer.com/player/cepede-royg/
https://www.tennisexplorer.com/player/niculescu/
https://www.tennisexplorer.com/player/giorgi/
https://www.tennisexplorer.com/player/parmentier/
https://www.tennisexplorer.com/player/larsson-7ecd5/
https://www.tennisexplorer.com/player/bouchard-d063c/
https://www.tennisexplorer.com/player/kozlova-3b58c/
https://www.tennisexplorer.com/player/brengle/
https://www.tennisexplorer.com/player/arruabarrena

...

https://www.tennisexplorer.com/player/giorgi/
https://www.tennisexplorer.com/player/razzano/
https://www.tennisexplorer.com/player/medina-garrigues/
https://www.tennisexplorer.com/player/kichenok-54bec/
https://www.tennisexplorer.com/player/arruabarrena-vecino/
https://www.tennisexplorer.com/player/duque-marino/
https://www.tennisexplorer.com/player/petrova/
https://www.tennisexplorer.com/player/dolonc/
https://www.tennisexplorer.com/player/minella/
https://www.tennisexplorer.com/player/martic/
https://www.tennisexplorer.com/player/pironkova/
https://www.tennisexplorer.com/player/putintseva/
https://www.tennisexplorer.com/player/lucic/
https://www.tennisexplorer.com/player/majeric/
https://www.tennisexplorer.com/player/van-uytvanck/
https://www.tennisexplorer.com/player/vandeweghe/
https://www.tennisexplorer.com/player/konta/
https://www.tennisexplorer.com/player/cabeza-candela/
https://www.tennisexplorer.com/player/fichman-0151f/
https://www.tennisexplorer.com/player/larcher-de-brito/

https://www.tennisexplorer.com/player/parmentier/
https://www.tennisexplorer.com/player/erakovic-d13eb/
https://www.tennisexplorer.com/player/babos-b2769/
https://www.tennisexplorer.com/player/vesnina/
https://www.tennisexplorer.com/player/king-6237c/
https://www.tennisexplorer.com/player/hampton/
https://www.tennisexplorer.com/player/kuznetsova/
https://www.tennisexplorer.com/player/rus-a3117/
https://www.tennisexplorer.com/player/larsson-7ecd5/
https://www.tennisexplorer.com/player/peer/
https://www.tennisexplorer.com/player/giorgi/
https://www.tennisexplorer.com/player/hercog/
https://www.tennisexplorer.com/player/minella/
https://www.tennisexplorer.com/player/beck-cee0d/
https://www.tennisexplorer.com/player/arruabarrena-vecino/
https://www.tennisexplorer.com/player/bratchikova/
https://www.tennisexplorer.com/player/benesova/
https://www.tennisexplorer.com/player/voskoboeva/
https://www.tennisexplorer.com/player/soler-espinosa/
https://www.tennisexplorer.com/player/oudin/
https://w

https://www.tennisexplorer.com/player/cetkovska/
https://www.tennisexplorer.com/player/kerber/
https://www.tennisexplorer.com/player/gajdosova/
https://www.tennisexplorer.com/player/kanepi/
https://www.tennisexplorer.com/player/martinez-sanchez-a21a4/
https://www.tennisexplorer.com/player/hercog/
https://www.tennisexplorer.com/player/peer/
https://www.tennisexplorer.com/player/scheepers/
https://www.tennisexplorer.com/player/pervak/
https://www.tennisexplorer.com/player/begu/
https://www.tennisexplorer.com/player/zakopalova/
https://www.tennisexplorer.com/player/paszek/
https://www.tennisexplorer.com/player/mchale/
https://www.tennisexplorer.com/player/strycova/
https://www.tennisexplorer.com/player/errani/
https://www.tennisexplorer.com/player/pironkova/
https://www.tennisexplorer.com/player/halep/
https://www.tennisexplorer.com/player/zheng-e622d/
https://www.tennisexplorer.com/player/morita-b536d/
https://www.tennisexplorer.com/player/martic/
https://www.tennisexplorer.com/player/ba

https://www.tennisexplorer.com/player/brengle/
https://www.tennisexplorer.com/player/bogdan-eb48d/
https://www.tennisexplorer.com/player/larriere/
https://www.tennisexplorer.com/player/mijacika/
https://www.tennisexplorer.com/player/wozniacki/
https://www.tennisexplorer.com/player/zvonareva/
https://www.tennisexplorer.com/player/clijsters/
https://www.tennisexplorer.com/player/williams-3d2a9/
https://www.tennisexplorer.com/player/williams/
https://www.tennisexplorer.com/player/stosur/
https://www.tennisexplorer.com/player/schiavone/
https://www.tennisexplorer.com/player/jankovic-aa821/
https://www.tennisexplorer.com/player/dementieva/
https://www.tennisexplorer.com/player/azarenka/
https://www.tennisexplorer.com/player/li-d1cc4/
https://www.tennisexplorer.com/player/henin-hardenne/
https://www.tennisexplorer.com/player/peer/
https://www.tennisexplorer.com/player/radwanska/
https://www.tennisexplorer.com/player/petrova/
https://www.tennisexplorer.com/player/bartoli/
https://www.tennisex

https://www.tennisexplorer.com/player/daniilidou/
https://www.tennisexplorer.com/player/groenefeld/
https://www.tennisexplorer.com/player/soler-espinosa/
https://www.tennisexplorer.com/player/foretz-gacon/
https://www.tennisexplorer.com/player/diatchenko/
https://www.tennisexplorer.com/player/irigoyen/
https://www.tennisexplorer.com/player/cohen-aloro/
https://www.tennisexplorer.com/player/diyas/
https://www.tennisexplorer.com/player/schruff/
https://www.tennisexplorer.com/player/garcia-vidagany/
https://www.tennisexplorer.com/player/lisicki/
https://www.tennisexplorer.com/player/watson-4ffc1/
https://www.tennisexplorer.com/player/linette/
https://www.tennisexplorer.com/player/de-los-rios/
https://www.tennisexplorer.com/player/o-brien-9fa28/
https://www.tennisexplorer.com/player/thorpe/
https://www.tennisexplorer.com/player/bychkova/
https://www.tennisexplorer.com/player/olaru/
https://www.tennisexplorer.com/player/rodionova-46097/
https://www.tennisexplorer.com/player/falconi-3f3f9/
h

# Injury_Record and Match_Record Table

In [168]:
# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Injury_Record;


CREATE TABLE Injury_Record (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    Players_id  INTEGER,
    start_date  TEXT,
    end_date   TEXT UNIQUE,
    tournament    TEXT,
    reason    TEXT
);

''')
        
cur.executescript('''
DROP TABLE IF EXISTS Match_Record;

CREATE TABLE Match_Record (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    Players_id  INTEGER,
    year INTEGER,
    tournament TEXT,
    match_type TEXT,
    date TEXT,
    opponent TEXT,
    round TEXT,
    result  TEXT,
    h FLOAT, 
    a FLOAT
);
''')

## injury_data Function: 
For each injury a player experiences, the following basic information is saved:<br>
start date, end date, tournament, reason
<br>
<br> __Input__ - The web-page from www.tennisexplorer.com
<br>__Output__ - Adding rows to the Injury_Record sqlite table

In [169]:
def injury_data(soup):
    for i, table in enumerate(soup.select("table")):
        # Injury Data
        if table["class"] == ["result", "flags", "injured"]:
            rows = table.find_all("tr")
            for row in rows:
                cols = row.find_all("td")
                cols = [x.text.strip() for x in cols]
                
                if len(cols) == 3:
                    dates = cols[0]
                    start_injury_date = dates.split(" - ")[0]
                    end_injury_date = dates.split(" - ")[1]
                    tournament = cols[1]
                    reason = cols[2]
                    name = soup.title # name
                    name = name.text[:-18]
                    cur.execute('SELECT id FROM Players WHERE name = ? ', (name, ))
                    Players_id = cur.fetchone()[0]
            
                    cur.execute('''INSERT OR IGNORE INTO Injury_Record 
                     ( Players_id, start_date, end_date, tournament, reason)
                     VALUES ( ?, ?, ?, ?, ?)''', 
                    (Players_id, str(start_injury_date), str(end_injury_date),
                    str(tournament), str(reason)))
                    conn.commit()
                else:
                    print("length is not 3")
            
            

## Match Results (tourn_res) Function: 
For each match a player plays, the following basic information is saved:<br>
year, tournament, match_type, date, opponent, round, result, h, a
<br>
<br> __Input__ - The web-page from tennisexplorer.com and year of interest
<br>__Output__ - Adding rows to the Match_Record sqlite table

In [170]:
# Extracting the Player Tournament Results
def tourn_res(soup):
    # iterate through different years of play
    for j in range(0, 13):
        year = 2020 - j
        # need to make sure that play took play that year
        # if exists, collect data, else == skip
        id_type = "matches-" + str(year) + "-1-data"
        test_exists = soup.find("div", {"id": id_type})
        if test_exists is None:
            print(str(year) + " Does not Exist!")
        else:
            # to iterate through all sing/dubs/mixed dub play in year
            for i in range(1, 4):
                id_type = "matches-" + str(year) + "-" + str(i) + "-data"
                table = soup.find("div", {"id": id_type})
                if i == 1:
                    mt = "singles"
                if i == 2:
                    mt = "doubles"
                if i == 3:
                    mt = "mixed doubles"

                rows = table.find_all("tr")
                for row in rows:
                    cols = row.find_all("td")
                    cols = [x.text.strip() for x in cols]
                    if len(cols) == 1:
                        tourn = cols[0]
                    else:
                        tournament = tourn
                        date = cols[0]
                        opponent = cols[2]
                        rounds = cols[3]
                        results = cols[4]
                        
                        h = cols[5]
                        try:
                            float(h)
                        except ValueError:
                            h = 0.0
                            
                        
                        a = cols[6]
                        try:
                            float(a)
                        except ValueError:
                            a = 0.0
                            
                        match_type = mt
                        year = year
                        # Saving info to table
                        name = soup.title # name
                        name = name.text[:-18]
                        cur.execute('SELECT id FROM Players WHERE name = ? ', (name, ))
                        Players_id = cur.fetchone()[0]

                        cur.execute('''INSERT OR IGNORE INTO Match_Record 
                         ( Players_id, year, tournament, match_type, date,
                         opponent, round, result, h, a)
                         VALUES ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', 
                            (Players_id, int(year), str(tournament),
                             str(match_type), str(date),opponent,str(rounds),str(results),float(h),float(a)))
                        conn.commit()
                

## Loop
This loop pulls out match and injury information from 2010 - 2020

In [ ]:
proxies = ['http://121.129.127.209:80', 'http://124.41.215.238:45169',
           'http://185.93.3.123:8080', 'http://194.182.64.67:3128', 
           'http://106.0.38.174:8080', 'http://163.172.175.210:3128',
           'http://13.92.196.150:8080']

for i in range(0, 11):
    '''
    Basic Data to save the match history for every player from 2010 - 2020 
    '''
    year = 2020 - i
    url_b = "https://www.tennisexplorer.com/ranking/wta-women/" + str(year)
    print(url_b)

    for i in range(1, 5):
        url = url_b + "/?page=" + str(i)
        print(url)
        proxy = {'http': random.choice(proxies)}
        
        with requests.get(url, proxies = proxy) as r:
            soup = BeautifulSoup(r.content, features="html")
            sl = random.randint(2, 4)
            time.sleep(sl)

        data = soup.find("tbody", {"class": "flags"})
        rows = data.find_all("td", {"class": "t-name"})

        for t in rows:
            player = t.find("a").attrs["href"]
            player_url = "https://www.tennisexplorer.com/" + player[1:]
            print(player_url)

            with requests.get(player_url, proxies = proxy) as r:
                soup = BeautifulSoup(r.content, features="html")
                sl = random.randint(2, 4)
                time.sleep(sl)

            # Find the summary hyperlink and pull out the injury
            # and sets/games played
            summary = soup.find("tr", {"class": "summary"})
            player = summary.find("a").attrs["href"]
            player_sum_url = "https://www.tennisexplorer.com/" + player[1:]
            print(player_sum_url)

            with requests.get(player_sum_url, proxies = proxy) as r:
                soup = BeautifulSoup(r.content, features="html")
                sl = random.randint(2, 4)
                time.sleep(sl)

            name_long = soup.title
            name = name_long.text[:-18]
            if name not in names:
                names.append(name)
                injury_data(soup)
                tourn_res(soup)

https://www.tennisexplorer.com/ranking/wta-women/2020
https://www.tennisexplorer.com/ranking/wta-women/2020/?page=1
https://www.tennisexplorer.com/player/barty/
https://www.tennisexplorer.com/player/barty/?annual=all
https://www.tennisexplorer.com/player/halep/
https://www.tennisexplorer.com/player/halep/?annual=all
https://www.tennisexplorer.com/player/pliskova/
https://www.tennisexplorer.com/player/pliskova/?annual=all
https://www.tennisexplorer.com/player/kenin-5a3a8/
https://www.tennisexplorer.com/player/kenin-5a3a8/?annual=all
https://www.tennisexplorer.com/player/svitolina/
https://www.tennisexplorer.com/player/svitolina/?annual=all
https://www.tennisexplorer.com/player/andreescu-281c7/
https://www.tennisexplorer.com/player/andreescu-281c7/?annual=all
https://www.tennisexplorer.com/player/bertens/
https://www.tennisexplorer.com/player/bertens/?annual=all
https://www.tennisexplorer.com/player/bencic/
https://www.tennisexplorer.com/player/bencic/?annual=all
https://www.tennisexplor

https://www.tennisexplorer.com/player/zhu-a98e2/?annual=all
https://www.tennisexplorer.com/player/cirstea/
https://www.tennisexplorer.com/player/cirstea/?annual=all
https://www.tennisexplorer.com/player/doi/
https://www.tennisexplorer.com/player/doi/?annual=all
https://www.tennisexplorer.com/player/flipkens/
https://www.tennisexplorer.com/player/flipkens/?annual=all
https://www.tennisexplorer.com/player/wang-64ea1/
https://www.tennisexplorer.com/player/wang-64ea1/?annual=all
https://www.tennisexplorer.com/player/brengle/
https://www.tennisexplorer.com/player/brengle/?annual=all
https://www.tennisexplorer.com/player/pegula/
https://www.tennisexplorer.com/player/pegula/?annual=all
https://www.tennisexplorer.com/player/begu/
https://www.tennisexplorer.com/player/begu/?annual=all
https://www.tennisexplorer.com/player/kuzmova/
https://www.tennisexplorer.com/player/kuzmova/?annual=all
https://www.tennisexplorer.com/player/sorribes-tormo/
https://www.tennisexplorer.com/player/sorribes-tormo/?

https://www.tennisexplorer.com/player/flink/
https://www.tennisexplorer.com/player/flink/?annual=all
https://www.tennisexplorer.com/player/gatto-monticone/
https://www.tennisexplorer.com/player/gatto-monticone/?annual=all
https://www.tennisexplorer.com/ranking/wta-women/2020/?page=4
https://www.tennisexplorer.com/player/trevisan-d1edd/
https://www.tennisexplorer.com/player/trevisan-d1edd/?annual=all
https://www.tennisexplorer.com/player/parmentier/
https://www.tennisexplorer.com/player/parmentier/?annual=all
https://www.tennisexplorer.com/player/cepede-royg/
https://www.tennisexplorer.com/player/cepede-royg/?annual=all
https://www.tennisexplorer.com/player/arruabarrena-vecino/
https://www.tennisexplorer.com/player/arruabarrena-vecino/?annual=all
https://www.tennisexplorer.com/player/kung-54014/
https://www.tennisexplorer.com/player/kung-54014/?annual=all
https://www.tennisexplorer.com/player/cocciaretto/
https://www.tennisexplorer.com/player/cocciaretto/?annual=all
https://www.tennisex

https://www.tennisexplorer.com/player/yastremska/?annual=all
https://www.tennisexplorer.com/player/sakkari/
https://www.tennisexplorer.com/player/sakkari/?annual=all
https://www.tennisexplorer.com/player/anisimova-3be62/
https://www.tennisexplorer.com/player/anisimova-3be62/?annual=all
https://www.tennisexplorer.com/player/stephens-82dfa/
https://www.tennisexplorer.com/player/stephens-82dfa/?annual=all
https://www.tennisexplorer.com/player/kontaveit/
https://www.tennisexplorer.com/player/kontaveit/?annual=all
https://www.tennisexplorer.com/player/sevastova/
https://www.tennisexplorer.com/player/sevastova/?annual=all
https://www.tennisexplorer.com/player/goerges/
https://www.tennisexplorer.com/player/goerges/?annual=all
https://www.tennisexplorer.com/player/collins-f1bd8/
https://www.tennisexplorer.com/player/collins-f1bd8/?annual=all
https://www.tennisexplorer.com/player/wang-ad6a3/
https://www.tennisexplorer.com/player/wang-ad6a3/?annual=all
https://www.tennisexplorer.com/player/pavly

https://www.tennisexplorer.com/player/paolini-4af25/?annual=all
https://www.tennisexplorer.com/player/badosa-gibert/
https://www.tennisexplorer.com/player/badosa-gibert/?annual=all
https://www.tennisexplorer.com/player/bogdan-947f3/
https://www.tennisexplorer.com/player/bogdan-947f3/?annual=all
https://www.tennisexplorer.com/player/stosur/
https://www.tennisexplorer.com/player/stosur/?annual=all
https://www.tennisexplorer.com/player/giorgi/
https://www.tennisexplorer.com/player/giorgi/?annual=all
https://www.tennisexplorer.com/ranking/wta-women/2019/?page=3
https://www.tennisexplorer.com/player/kanepi/
https://www.tennisexplorer.com/player/kanepi/?annual=all
https://www.tennisexplorer.com/player/begu/
https://www.tennisexplorer.com/player/begu/?annual=all
https://www.tennisexplorer.com/player/watson-4ffc1/
https://www.tennisexplorer.com/player/watson-4ffc1/?annual=all
https://www.tennisexplorer.com/player/gasparyan-e426d/
https://www.tennisexplorer.com/player/gasparyan-e426d/?annual=al

...

https://www.tennisexplorer.com/player/korpatsch/?annual=all
https://www.tennisexplorer.com/player/rus-a3117/
https://www.tennisexplorer.com/player/rus-a3117/?annual=all
https://www.tennisexplorer.com/player/kostyuk-ea2bf/
https://www.tennisexplorer.com/player/kostyuk-ea2bf/?annual=all
https://www.tennisexplorer.com/player/zhu-a98e2/
https://www.tennisexplorer.com/player/zhu-a98e2/?annual=all
https://www.tennisexplorer.com/player/stollar/
https://www.tennisexplorer.com/player/stollar/?annual=all
https://www.tennisexplorer.com/player/wickmayer/
https://www.tennisexplorer.com/player/wickmayer/?annual=all
https://www.tennisexplorer.com/player/gibbs-a7e35/
https://www.tennisexplorer.com/player/gibbs-a7e35/?annual=all
https://www.tennisexplorer.com/player/glushko/
https://www.tennisexplorer.com/player/glushko/?annual=all
https://www.tennisexplorer.com/player/peng/
https://www.tennisexplorer.com/player/peng/?annual=all
https://www.tennisexplorer.com/player/dolehide-903f7/
https://www.tennisex

https://www.tennisexplorer.com/player/paolini-4af25/
https://www.tennisexplorer.com/player/paolini-4af25/?annual=all
https://www.tennisexplorer.com/player/di-giuseppe/
https://www.tennisexplorer.com/player/di-giuseppe/?annual=all
https://www.tennisexplorer.com/player/lottner/
https://www.tennisexplorer.com/player/lottner/?annual=all
https://www.tennisexplorer.com/player/pattinama-kerkhove/
https://www.tennisexplorer.com/player/pattinama-kerkhove/?annual=all
https://www.tennisexplorer.com/player/shimizu-f5d8f/
https://www.tennisexplorer.com/player/shimizu-f5d8f/?annual=all
https://www.tennisexplorer.com/player/martincova-a08dc/
https://www.tennisexplorer.com/player/martincova-a08dc/?annual=all
https://www.tennisexplorer.com/player/muhammad-c4e56/
https://www.tennisexplorer.com/player/muhammad-c4e56/?annual=all
https://www.tennisexplorer.com/player/lisicki/
https://www.tennisexplorer.com/player/lisicki/?annual=all
https://www.tennisexplorer.com/player/ahn-a253d/
https://www.tennisexplore

https://www.tennisexplorer.com/player/brady-cbcc6/
https://www.tennisexplorer.com/player/brady-cbcc6/?annual=all
https://www.tennisexplorer.com/player/diyas/
https://www.tennisexplorer.com/player/diyas/?annual=all
https://www.tennisexplorer.com/player/vondrousova/
https://www.tennisexplorer.com/player/vondrousova/?annual=all
https://www.tennisexplorer.com/player/osaka-fa603/
https://www.tennisexplorer.com/player/osaka-fa603/?annual=all
https://www.tennisexplorer.com/player/siegemund/
https://www.tennisexplorer.com/player/siegemund/?annual=all
https://www.tennisexplorer.com/player/riske-69d46/
https://www.tennisexplorer.com/player/riske-69d46/?annual=all
https://www.tennisexplorer.com/player/haddad-maia/
https://www.tennisexplorer.com/player/haddad-maia/?annual=all
https://www.tennisexplorer.com/player/linette/
https://www.tennisexplorer.com/player/linette/?annual=all
https://www.tennisexplorer.com/player/sabalenka/
https://www.tennisexplorer.com/player/sabalenka/?annual=all
https://www

https://www.tennisexplorer.com/player/blinkova/
https://www.tennisexplorer.com/player/blinkova/?annual=all
https://www.tennisexplorer.com/player/teichmann/
https://www.tennisexplorer.com/player/teichmann/?annual=all
https://www.tennisexplorer.com/player/jang-11db2/
https://www.tennisexplorer.com/player/jang-11db2/?annual=all
https://www.tennisexplorer.com/player/martincova-a08dc/
https://www.tennisexplorer.com/player/martincova-a08dc/?annual=all
https://www.tennisexplorer.com/player/voegele/
https://www.tennisexplorer.com/player/voegele/?annual=all
https://www.tennisexplorer.com/player/minella/
https://www.tennisexplorer.com/player/minella/?annual=all
https://www.tennisexplorer.com/player/zhao-af061/
https://www.tennisexplorer.com/player/zhao-af061/?annual=all
https://www.tennisexplorer.com/player/loeb/
https://www.tennisexplorer.com/player/loeb/?annual=all
https://www.tennisexplorer.com/player/liu-82232/
https://www.tennisexplorer.com/player/liu-82232/?annual=all
https://www.tennisexp

https://www.tennisexplorer.com/player/azarenka/
https://www.tennisexplorer.com/player/azarenka/?annual=all
https://www.tennisexplorer.com/player/svitolina/
https://www.tennisexplorer.com/player/svitolina/?annual=all
https://www.tennisexplorer.com/player/bacsinszky/
https://www.tennisexplorer.com/player/bacsinszky/?annual=all
https://www.tennisexplorer.com/player/vesnina/
https://www.tennisexplorer.com/player/vesnina/?annual=all
https://www.tennisexplorer.com/player/williams/
https://www.tennisexplorer.com/player/williams/?annual=all
https://www.tennisexplorer.com/player/vinci/
https://www.tennisexplorer.com/player/vinci/?annual=all
https://www.tennisexplorer.com/player/wozniacki/
https://www.tennisexplorer.com/player/wozniacki/?annual=all
https://www.tennisexplorer.com/player/strycova/
https://www.tennisexplorer.com/player/strycova/?annual=all
https://www.tennisexplorer.com/player/stosur/
https://www.tennisexplorer.com/player/stosur/?annual=all
https://www.tennisexplorer.com/player/ber

# Rank Table

In [161]:
cur.executescript('''
DROP TABLE IF EXISTS Rank;

CREATE TABLE Rank (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    Players_id  INTEGER,
    year INTEGER,
    ranktype TEXT,
    rank INTEGER
);

''')

## Ranking by Year (Rank) Function: 
The top 200 players from 2010 - 2020.<br>
Players_id, year, p_rank_type, rank
<br>
<br> __Input__ - The web-page from tennisexplorer.com
<br>__Output__ - Adding rows to the Rank sqlite table

In [159]:

# Replaced 0 
for i in range(0, 11):
    year = 2020 - i
    url_b = "https://www.tennisexplorer.com/ranking/wta-women/" + str(year)
    print(url_b)

    # Singles Rankings
    for i in range(1, 5):
        p_rank_type = "Singles"
        url = url_b + "/?page=" + str(i)
        print(url)
        proxy = {'http': random.choice(proxies)}
        with requests.get(url, proxies = proxy) as r:
            soup = BeautifulSoup(r.content, features="html")
            sl = random.randint(3, 5)
            time.sleep(sl)
            data = soup.find("tbody", {"class": "flags"})
            rows_name = data.find_all("td", {"class": "t-name"})
            rows_rank = data.find_all("td", {"class": "rank first"})

            for i in range(0, len(rows_rank)):
                player_row = rows_name[i]
                player = player_row.string
                
                # Edge Cases (18 total)
                if player == 'Van Uytvanck Alison':
                    player = 'Alison Van Uytvanck'                
                elif player == 'Sorribes Tormo Sara':
                    player = 'Sara Sorribes Tormo'                
                elif player == 'Badosa Gibert Paula':
                    player = 'Paula Badosa Gibert'                
                elif player == 'Di Lorenzo Francesca':
                    player = 'Francesca Di Lorenzo'                
                elif player == 'Cepede Royg Veronica':
                    player = 'Veronica Cepede Royg'                
                elif player == 'Pattinama Kerkhove Lesley':
                    player = 'Lesley Pattinama Kerkhove'                
                elif player == 'De Vroome Indy':
                    player = 'Indy De Vroome'                
                elif player == 'Di Giuseppe Martina':
                    player  = 'Martina Di Giuseppe'                    
                elif player == 'Haddad Maia Beatriz':
                    player = 'Beatriz Haddad Maia'                    
                elif player == 'Von Deichmann Kathinka':
                    player = 'Kathinka Von Deichmann'                    
                elif player == 'Jovanovski Petrovic Bojana':
                    player = 'Bojana Jovanovski Petrovic'                    
                elif player == 'Larcher de Brito Michelle':
                    player = 'Michelle Larcher de Brito'                    
                elif player == 'Foretz Gacon Stephanie':
                    player = 'Stephanie Foretz Gacon'                    
                elif player == 'Medina Garrigues Anabel':
                    player = 'Anabel Medina Garrigues'                    
                elif player == 'El Tabakh Heidi':
                    player = 'Heidi El Tabakh'                    
                elif player == 'Martinez Sanchez Maria Jose':
                    player = 'Maria Jose Martinez Sanchez'
                elif player == 'Llagostera Vives Nuria':
                    player = 'Nuria Llagostera Vives' 
                elif player == 'De Los Rios Rosana':
                    player = 'Rosana De Los Rios'
                
                # Otherwise use normal seperator
                else:
                    # Need to flip for first and last name
                    f_name = player.split()[1:]
                    f_name_f = ' '.join(f_name)
                    l_name = player.split()[0]
                    player = f_name_f + ' ' + l_name
                

                ranking_row = rows_rank[i]
                rank = int(float(ranking_row.string))
                
                # Adding Row of information to the table
                cur.execute('SELECT id FROM Players WHERE name = ? ', (player, ))
                Players_id = cur.fetchone()[0]
                
                cur.execute('''INSERT OR IGNORE INTO Rank 
                ( Players_id, year, ranktype, rank)
                VALUES ( ?, ?, ?, ?)''', 
                (Players_id, year, p_rank_type, rank))
                conn.commit()   
            
    # Doubles Rankings
    for i in range(1, 5):
        url = url_b + "/?t=doubles&page=" + str(i)
        print(url)
        p_rank_type = "Doubles"
        proxy = {'http': random.choice(proxies)}

        with requests.get(url,proxies = proxy) as r:
            soup = BeautifulSoup(r.content, features="html")
            sl = random.randint(3, 5)
            time.sleep(sl)
            data = soup.find("tbody", {"class": "flags"})
            rows_name = data.find_all("td", {"class": "t-name"})
            rows_rank = data.find_all("td", {"class": "rank first"})


            for i in range(0, len(rows_rank)):
                player_row = rows_name[i]
                player = player_row.string


                # Edge Cases (18 total)
                if player == 'Van Uytvanck Alison':
                    player = 'Alison Van Uytvanck'                
                elif player == 'Sorribes Tormo Sara':
                    player = 'Sara Sorribes Tormo'                
                elif player == 'Badosa Gibert Paula':
                    player = 'Paula Badosa Gibert'                
                elif player == 'Di Lorenzo Francesca':
                    player = 'Francesca Di Lorenzo'                
                elif player == 'Cepede Royg Veronica':
                    player = 'Veronica Cepede Royg'                
                elif player == 'Pattinama Kerkhove Lesley':
                    player = 'Lesley Pattinama Kerkhove'                
                elif player == 'De Vroome Indy':
                    player = 'Indy De Vroome'                
                elif player == 'Di Giuseppe Martina':
                    player  = 'Martina Di Giuseppe'                    
                elif player == 'Haddad Maia Beatriz':
                    player = 'Beatriz Haddad Maia'                    
                elif player == 'Von Deichmann Kathinka':
                    player = 'Kathinka Von Deichmann'                    
                elif player == 'Jovanovski Petrovic Bojana':
                    player = 'Bojana Jovanovski Petrovic'                    
                elif player == 'Larcher de Brito Michelle':
                    player = 'Michelle Larcher de Brito'                    
                elif player == 'Foretz Gacon Stephanie':
                    player = 'Stephanie Foretz Gacon'                    
                elif player == 'Medina Garrigues Anabel':
                    player = 'Anabel Medina Garrigues'                    
                elif player == 'El Tabakh Heidi':
                    player = 'Heidi El Tabakh'                    
                elif player == 'Martinez Sanchez Maria Jose':
                    player = 'Maria Jose Martinez Sanchez'
                elif player == 'Llagostera Vives Nuria':
                    player = 'Nuria Llagostera Vives' 
                elif player == 'De Los Rios Rosana':
                    player = 'Rosana De Los Rios'
                
                
                # Otherwise use normal seperator
                else:
                    # Need to flip for first and last name
                    f_name = player.split()[1:]
                    f_name_f = ' '.join(f_name)
                    l_name = player.split()[0]
                    player = f_name_f + ' ' + l_name
                    
                ranking_row = rows_rank[i]
                rank = int(float(ranking_row.string))

                cur.execute('SELECT id FROM Players WHERE name = ? ', (player, ))

                x = cur.fetchone()
                
                if not x:
                    Players_id = None
                else:
                    Players_id = x[0]

                
                cur.execute('''INSERT OR IGNORE INTO Rank 
                ( Players_id, year, ranktype, rank)
                VALUES ( ?, ?, ?, ?)''', 
                (Players_id, year, p_rank_type, rank))
                conn.commit()   
                

https://www.tennisexplorer.com/ranking/wta-women/2020
https://www.tennisexplorer.com/ranking/wta-women/2020/?page=1
https://www.tennisexplorer.com/ranking/wta-women/2020/?page=2
https://www.tennisexplorer.com/ranking/wta-women/2020/?page=3
https://www.tennisexplorer.com/ranking/wta-women/2020/?page=4
https://www.tennisexplorer.com/ranking/wta-women/2020/?t=doubles&page=1
https://www.tennisexplorer.com/ranking/wta-women/2020/?t=doubles&page=2
https://www.tennisexplorer.com/ranking/wta-women/2020/?t=doubles&page=3
https://www.tennisexplorer.com/ranking/wta-women/2020/?t=doubles&page=4
https://www.tennisexplorer.com/ranking/wta-women/2019
https://www.tennisexplorer.com/ranking/wta-women/2019/?page=1
https://www.tennisexplorer.com/ranking/wta-women/2019/?page=2
https://www.tennisexplorer.com/ranking/wta-women/2019/?page=3
https://www.tennisexplorer.com/ranking/wta-women/2019/?page=4
https://www.tennisexplorer.com/ranking/wta-women/2019/?t=doubles&page=1
https://www.tennisexplorer.com/ranki

# Cout_Type Table
* Unique id
* Tournament
* Court Type

__Based off of database__
Database information is downloaded as csv files

In [ ]:
cur.executescript('''
DROP TABLE IF EXISTS Cout_Type;

CREATE TABLE Cout_Type (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    tournament  TEXT UNIQUE
    surface     TEXT
);

''')

In [ ]:
from os import listdir

# Change the Working Directory
os.chdir("/Users/Owner/Desktop/InsightFellows/Daniella_Patton_Insight_Project/tennis_wta")

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]


filenames = find_csv_filenames("my/directory")

for file in filenames:
    print(file)
    df = pd.read_csv(file)
    tournaments = df.tourney_name
    surfaces = df.surface
    
    for i in range(0, len(df)):
        tournament = tournaments[i]
        surface = surfaces[i]
        
        cur.execute('''INSERT OR IGNORE INTO Cout_Type 
                ( tournament, surface)
                VALUES ( ?, ?)''', 
                (tournament, surface))
        conn.commit()   